# House Price Prediction Project

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('Downloads/kc_house_data.csv')

# Display basic information
print(data.info())
print(data.describe())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21611 non-null  float64
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [2]:
# Handle missing values
data = data.dropna()

# Remove duplicates
data = data.drop_duplicates()

# Convert categorical columns to numerical using one-hot encoding
data = pd.get_dummies(data, columns=['zipcode'], drop_first=True)

print(data.info())


<class 'pandas.core.frame.DataFrame'>
Index: 21611 entries, 0 to 21612
Data columns (total 89 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21611 non-null  int64  
 1   date           21611 non-null  object 
 2   price          21611 non-null  float64
 3   bedrooms       21611 non-null  int64  
 4   bathrooms      21611 non-null  float64
 5   sqft_living    21611 non-null  int64  
 6   sqft_lot       21611 non-null  int64  
 7   floors         21611 non-null  float64
 8   waterfront     21611 non-null  int64  
 9   view           21611 non-null  int64  
 10  condition      21611 non-null  int64  
 11  grade          21611 non-null  int64  
 12  sqft_above     21611 non-null  float64
 13  sqft_basement  21611 non-null  int64  
 14  yr_built       21611 non-null  int64  
 15  yr_renovated   21611 non-null  int64  
 16  lat            21611 non-null  float64
 17  long           21611 non-null  float64
 18  sqft_living

In [3]:
# Feature engineering: Create new features
data['age'] = 2024 - data['yr_built']
data['renovated'] = (data['yr_renovated'] > 0).astype(int)

# Select features and target variable
features = data.drop(columns=['price', 'date', 'id', 'yr_built', 'yr_renovated'])
target = data['price']

print(features.head())
print(target.head())


   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0         3       1.00         1180      5650     1.0           0     0   
1         3       2.25         2570      7242     2.0           0     0   
2         2       1.00          770     10000     1.0           0     0   
3         4       3.00         1960      5000     1.0           0     0   
4         3       2.00         1680      8080     1.0           0     0   

   condition  grade  sqft_above  ...  zipcode_98155  zipcode_98166  \
0          3      7      1180.0  ...          False          False   
1          3      7      2170.0  ...          False          False   
2          3      6       770.0  ...          False          False   
3          5      7      1050.0  ...          False          False   
4          3      8      1680.0  ...          False          False   

   zipcode_98168  zipcode_98177  zipcode_98178  zipcode_98188  zipcode_98198  \
0          False          False           True  

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)


In [5]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 18662193337.724213
R^2 Score: 0.8750868545685662


In [6]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict and evaluate the optimized model
y_pred_optimized = best_model.predict(X_test)
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

print(f'Optimized Mean Squared Error: {mse_optimized}')
print(f'Optimized R^2 Score: {r2_optimized}')


Optimized Mean Squared Error: 18398552543.306995
Optimized R^2 Score: 0.8768515025013537


In [7]:
import joblib

# Save the model
joblib.dump(best_model, 'home_value_model.pkl')

# Load the model
loaded_model = joblib.load('home_value_model.pkl')

# Use the loaded model for predictions
new_predictions = loaded_model.predict(X_test)
